### Sample program for SVR fitting and saving model file  

#### Import libraries  

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from joblib import dump

#### Parameters  

In [ ]:
csv_in = 'vibration.csv'

#### Read CSV file  

In [ ]:
df = pd.read_csv(csv_in, delimiter=',', skiprows=0, header=0)
print(df.shape)
print(df.info())
display(df.head())

#### Obtain X and y  

In [ ]:
X = df[['x']]
print(X.shape)
y = df['y']
print(y.shape)

#### Split X and y for train and test  

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1)

#### Scaling (NOTE: sc is generated by training data)  

In [ ]:
sc = MinMaxScaler(feature_range=(0, 1), copy=True)
sc.fit(X_train)
X_train_std = sc.transform(X_train)

#### Parameters for SVR  

In [ ]:
C = 100.0
gamma = 'scale'
epsilon = 0.2

#### Execute SVR  

In [ ]:
svr = SVR(gamma=gamma, C=C, epsilon=epsilon)
svr.fit(X_train_std, y_train)

#### Do prediction using train data  

In [ ]:
y_train_pred = svr.predict(X_train_std)

#### Scaling of Test data and do prediction  

In [ ]:
X_test_std = sc.transform(X_test)
y_test_pred = svr.predict(X_test_std)

#### Check MSE  

In [ ]:
mse_train = mean_squared_error(y_train_pred, y_train)
print('MSE (train):', mse_train)
mse_test = mean_squared_error(y_test_pred, y_test)
print('MSE (test):', mse_test)

#### Scatter plot of the prediction of SVR and true value for train / test data  

In [ ]:
plt.scatter(X_train_std, y_train_pred, label='train_pred')
plt.scatter(X_train_std, y_train, label='train_true')
plt.legend()
plt.xlabel('X')
plt.ylabel('y')
plt.show()

plt.scatter(X_test_std, y_test_pred, label='test_pred')
plt.scatter(X_test_std, y_test, label='test_true')
plt.legend()
plt.xlabel('X')
plt.ylabel('y')
plt.show()

#### Scatter plot of output of SVR (prediction) and true value for train / test data  

In [ ]:
plt.scatter(y_train_pred, y_train, label='train')
xmin = np.array([y_train.min(), y_train.max()])
xmax = np.array([y_train.min(), y_train.max()])
plt.plot(xmin, xmax, label='y=x')
plt.xlabel('Pred')
plt.ylabel('True')
plt.legend()
plt.show()

plt.scatter(y_test_pred, y_test, label='test')
xmin = np.array([y_train_pred.min(), y_train_pred.max()])
xmax = np.array([y_train_pred.min(), y_train_pred.max()])
plt.plot(xmin, xmax, label='y=x')
plt.xlabel('Pred')
plt.ylabel('True')
plt.legend()
plt.show()

#### Save developed SVR model  

In [ ]:
tag = 'vibration'
sc_file = 'sc_{}.joblib'.format(tag)
dump(sc, sc_file)
model_file = 'svr_{}_rbf_c{}_g{}_e{}.joblib'.format(tag,C,gamma,epsilon)
dump(svr, model_file)